In [0]:
import pandas as pd
import numpy as np
import os

path = '/home/danielipanao/'
df_data = pd.read_excel(
     os.path.join(path, "Downloads", "HND_DATA.xlsm"),
     engine='openpyxl',
     sheet_name='DATA'
)

df_data = df_data[df_data["Torre"].notna()]
df_data = df_data[df_data["Tipo Servico"].isin(["INC", "RITM"])]
df_data = df_data[(df_data["Estado"] != "Cancelado") & (df_data["Estado"] != "Aguardando Cancelamento")]
df_data = df_data[df_data["Torre"].isin(["Honda Servicos Financeiros", "Honda South America"])]
df_data["Ano Abertura"] = df_data["Ano Abertura"].astype(int)
df_data["Mes Abertura"] = df_data["Mes Abertura"].astype(int)
df_data = df_data[df_data["Mes Abertura"] == 5]

df_data["Res_Mesmo_Mes"] = np.where((df_data["Mes Abertura"] == df_data["Mes Resolvido"])
                                    & (df_data["Ano Abertura"] == df_data["Ano Resolvido"]), 'S', 'N')

df_data_qtde_total = df_data.groupby(["Tipo Servico", "Torre", "Mes Abertura", "Ano Abertura", "Mes Abertura | Ano"])\
                            .size().reset_index(name='Qtde_Total')

df_data_resolvido = df_data[df_data["Res_Mesmo_Mes"] == "S"]
df_data_qtde_resolvido = df_data_resolvido.groupby(["Tipo Servico", "Torre", "Mes Abertura", "Ano Abertura"
                                                   ,"Mes Abertura | Ano"])\
                        .size().reset_index(name='Qtde_Resolvido')

df_join = df_data_qtde_total.merge (df_data_qtde_resolvido,
                                    on = ['Torre', 'Tipo Servico', 'Mes Abertura', 'Ano Abertura', 'Mes Abertura | Ano']
                                    , how = 'left')
df_join = df_join.fillna(0)

df_join["Qtde_Total"] = df_join["Qtde_Total"].astype(int)
df_join["Qtde_Resolvido"] = df_join["Qtde_Resolvido"].astype(int)


df_join = df_join[["Tipo Servico", "Torre", "Mes Abertura", "Ano Abertura", "Qtde_Resolvido", "Qtde_Total",
                   "Mes Abertura | Ano"]]

df_join["% Outflow"] = df_join["Qtde_Resolvido"] * 100 / df_join["Qtde_Total"]
df_join["% Outflow"] = df_join["% Outflow"].astype(int)

df_join = df_join[["Tipo Servico", "Torre", "Mes Abertura", "Ano Abertura", "Mes Abertura | Ano", "% Outflow"]]

print(df_join)
df_join.to_excel('/home/danielipanao/Downloads/teste.xlsx', index=False)